In [2]:
import pandas as pd

In [3]:
df = pd.read_table('../data/payments_training.tsv').drop(columns=['1'])
df.columns = ['date', 'amount', 'text', 'class']

In [4]:
df[:400].to_csv('../data/dataset_train.csv')
df[400:].to_csv('../data/dataset_test.csv')

In [3]:
df['class'] = pd.Categorical(df['class'])
df.amount = df.amount.apply(lambda x: float(x.replace(',', '.').replace('-', '.')))

In [4]:
df.head(3)

,date,amount,text,class
0,07.11.2024,40200.0,За оказание услуг по договору №79-02726В от 01...,SERVICE
1,07.11.2024,1440.0,Оплата за Порошок стиральный Ariel Color autom...,NON_FOOD_GOODS
2,07.11.2024,240000000.0,Возврат денежных средств по договору займа №04...,LOAN


## Очистка датасета

In [5]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

only_text_regex = r'[^а-яА-ЯёЁ]+'
stop_words = set(stopwords.words('russian'))


def clean_text(text):
    text = re.sub(only_text_regex, ' ', text)
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    
    return text

[nltk_data] Downloading package stopwords to /home/misha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
clean_text(df.text[0])

'оказание услуг договору г'

In [7]:
df['clean_text'] = df['text'].apply(clean_text)
df['clean_text']

0                              оказание услуг договору г
1      Оплата Порошок стиральный кг счету августа г С...
2            Возврат денежных средств договору займа НДС
3      Оплата Дог согл Сч г оплата сброс загрязняющих...
4      Оплата дог финансовой аренды Б г акт приема пе...
                             ...                        
494                                       Оплата налогов
495    Комиссия выполнение функций агента валютного к...
496                               тур поездку договору Г
497    Оплата договору г счету г Дымовские колбасы Су...
498                            Оплата коммунальных услуг
Name: clean_text, Length: 499, dtype: object

# Наивный байесовский классификатор

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score, f1_score

In [9]:
vectorizer = TfidfVectorizer()

### На грязном тексте

In [10]:
X = vectorizer.fit_transform(df.text.tolist())
y = df['class']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [13]:
y_pred = nb_model.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'F1-Score: {f1_score(y_test, y_pred, average="macro")}')

Accuracy: 0.86
F1-Score: 0.8381165895072613


### На чистом тексте

In [14]:
X = vectorizer.fit_transform(df.clean_text.tolist())
y = df['class']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [17]:
y_pred = nb_model.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'F1-Score: {f1_score(y_test, y_pred, average="macro")}')

Accuracy: 0.91
F1-Score: 0.9128021321569708


# FastText

In [2]:
0.913-0.84

0.07300000000000006

In [25]:
import fasttext

формат текста для FastText

In [26]:
df['formatted_class'] = df['class'].apply(lambda x: f'__label__{x}')
df[['clean_text', 'formatted_class']].to_csv('fasttext_data.txt', index=False, header=False)

In [27]:
model = fasttext.train_supervised(input='fasttext_data.txt', label='__label__', epoch=25, lr=0.01, dim=50)

Read 0M words
Number of words:  697
Number of labels: 0
Progress: 100.0% words/sec/thread:  349020 lr:  0.000000 avg.loss:      -nan ETA:   0h 0m 0s


In [121]:
model.predict(df.iloc[0].clean_text)

((), array([], dtype=float64))